In [1]:
import pandas as pd

In [2]:
def read_files(file_date):
    '''
    Import 3 data files, base data B,C & team structure file.

    B (download from CRM) mainly contains Building Info, Agents Info & Financial Info 
    C (download from CRM) mainly contains Customer Info & Commission Info
    Peroneel is a excel records the team structure updated by HR
    
    CRM data may contains test deals(registrant info contains 'test'), deleted them first
    
    Args:
        file_date: data sets downloaded from CRM are formatted like 'mm-dd-B/C.xlsx'
    
    Returns:
        agent_level: Team structure in DataFrame
        main_sheet_b: Raw data B in DataFrame
        main_sheet_c: Raw data C in DataFrame
    '''
    b_name = file_date + '-B.xlsx'
    c_name = file_date + '-C.xlsx'
    
    agent_level = pd.read_excel('team structure.xlsx',parse_dates=True).sort_index() 
    
    main_sheet_c = pd.read_excel(c_name,parse_dates=True).sort_index()
    main_sheet_c = main_sheet_c[~main_sheet_c['registrant'].str.contains('test')]

    id_need_deleted = main_sheet_c[main_sheet_c['registrant'].str.contains('test')]["DealID"].to_list()
    
    main_sheet_b = pd.read_excel(b_name,parse_dates=True).drop_duplicates().reset_index(drop = True)
    main_sheet_b = main_sheet_b[~main_sheet_b['DealID'].isin(id_need_deleted)]
    
    return agent_level, main_sheet_b, main_sheet_c

In [3]:
def date_fix(df,column):
    '''
    Set Date Info from CRM data files to proper date format
    
    Args:
        df: which DataFrame need to be changed
        column: which column need to be changed
        
    Returns:
        No return needed since this function is updating DataFrame
    '''
    df[column] = pd.to_datetime(df[column].str.split(' ',1).str[0])

In [4]:
def period(main_sheet_c,time_start,time_end):
    '''Slicing C for certain period
    
    Args:
        main_sheet_c: Raw data file C
        time_start: Start date of the period, inclusively
        time_end: End date of the period, inclusively
        
    Returns:
        time_range: Certain period of data from C in DataFrame
    '''
    
    time_range = main_sheet_c[(main_sheet_c['DealDate'] >= time_start) \
                              &(main_sheet_c['DealDate'] <= time_end)]
    return time_range

In [5]:
def deals_status(status,time_range):
    '''Specify what kind of deals we want to calculate
    
    All deals are taged with one of the three status: pending, closed, dropped
    
    Args:
        status: 'Pending + Closed', calculate all deals except Dropped, default
                'Pending', calculate only pending deals
                'Closed', calculate only closed deals
                'Dropped', calculate only dropped deals
    
    Returns:
        No return needed since this function updating time_range DataFrame directly
        
        
    C contains data from different versions of CRM, the spelling of status varies, using
    contains rather than ==
    '''
    
    if status == 'Pending + Closed':
        time_range = time_range[~time_range['Status'].str.contains('rop', na=False)]
    elif status == 'Pending':
        time_range = time_range[time_range['Status'].str.contains('ending')]
    elif status == 'Closed':
        time_range = time_range[time_range['Status'].str.contains('lose')]
    elif status == 'Dropped':
        time_range = time_range[time_range['Status'].str.contains('rop')]
    

In [6]:
# pandas.DataFrame.apply() functions for mapping supervisors for every deal, add a new column for each row
# TL column: TL name for this deal
# M columns: Manager name for this deal
# Senior Agent: Senior Agent anem for this deal
# Each row(deal) must have TL name, but Manager and Senior Agent info are not mandantory

def TL_match(x,agent_level):
    '''
    DataFrame.apply() function for mapping TL
    
    Args:
        x: DataFrame.apply() function default arg represents each row
        agent_level: Team structure DataFrame
    
    Returns:
        TL info
    '''
    supervisor = agent_level[(agent_level['Agent Name'] == x['registrant']) & \
                       (agent_level['T Start'] <= x['DealDate']) & \
                       (agent_level['T End'] >= x['DealDate'])]
    return supervisor['TL'][0:1].to_string(index = False)
    



def M_match(x,agent_level):
    '''
    DataFrame.apply() function for mapping Manager
    
    Args:
        x: DataFrame.apply() function default arg represents each row
        agent_level: Team structure DataFrame
    
    Returns:
        Manager info
    '''
    supervisor = agent_level[(agent_level['Agent Name'] == x['registrant']) & \
                       (agent_level['M Start'] <= x['DealDate']) & \
                       (agent_level['M End'] >= x['DealDate'])]
    return supervisor['Manager'].to_string(index = False)



def SA_match(x,agent_level):
    '''
    DataFrame.apply() function for mapping Senior Agent
    
    Args:
        x: DataFrame.apply() function default arg represents each row
        agent_level: Team structure DataFrame
    
    Returns:
        Senior Agent info
    '''
    supervisor = agent_level[(agent_level['Agent Name'] == x['registrant']) & \
                       (agent_level['SA Start'] <= x['DealDate']) & \
                       (agent_level['SA End'] >= x['DealDate'])]
    return supervisor['Senior Agent'].to_string(index = False)

In [7]:
def ranking(time_range_tag):
    '''Calculate commission of each agent, manager team and team leader group
    
    Args:
        time_range_tag: Data from C in certain staus and period
    
    Returns:
        agent_rank: Commission ranking for each agent in discending order
        manager_rank: Commission ranking for each manager team in discending order
        tl_rank: Commission ranking for each team leader team in discending order
    
    '''
    agent_rank = time_range_tag.groupby('registrant').Comm.sum().sort_values(ascending = False)

    manager_rank = time_range_tag.groupby('M').Comm.sum().sort_values(ascending = False)

    tl_rank = time_range_tag.groupby('TL').Comm.sum().sort_values(ascending = False)
    
    return agent_rank,manager_rank,tl_rank


In [8]:
def mapping_building(time_range_tag,main_sheet_b):
    '''Mapping building info for each row(deal)
    
    Each row(deal) from C mianly contains customer info and commission info, we need to map the 
    Building info and other info from B to C
    
    Args:
        time_range_tag: DataFrame from C in certain status and period with Supervisor info\
        main_sheet_b: Raw data B in DataFrame
    
    Returns:
        merged: data C mapped with info from B
    '''
    
    the_columns = ['Building','Apt','BalAndBoth','Rent','BrokerLicsences','LeaseStartDate']

    merged = pd.concat([time_range_tag,pd.DataFrame(columns = the_columns)], sort = False)

    for i in merged['DealID'].unique():
        for p in merged[merged['DealID'] == i].index:
            sheet_b_index = main_sheet_b[main_sheet_b['DealID'] == i].index[0]
            merged.loc[p,the_columns] = main_sheet_b.loc[sheet_b_index,the_columns]
    
    return merged

In [9]:
def merging_cap(merged):
    '''Combine superviors, rent, deal date etc. info for same deal
    
    Capitalize Apt 
    One deal could be done by different agents, and each agent may have different supervisors
    For same deal, different agents may finish it in different date with different commission
    Combine those info for the same deal
    Manager, TL: Combine unique names
    DealDate, LeaseStart Date: Keep the smallest date
    Rent, B&B: Keep the Smallest value
    
    Args:
        merged: data C mapped with info from B
    
    Returns:
        No return needed, DataFrame updating
    ''' 
    
    merged['Apt'] = merged['Apt'].str.upper()

    merged['Agent_Combined'] = merged.groupby(['Building','Apt'])['registrant'].transform('unique').str.join('/')

    merged['Manager_Combined'] = merged.groupby(['Building','Apt'])['M'].transform('unique').str.join('/')

    merged['TL_Combined'] = merged.groupby(['Building','Apt'])['TL'].transform('unique').str.join('/')

    merged['DealDate'] = merged.groupby(['Building','Apt'])['DealDate'].transform('min')

    merged['LeaseStartDate'] = merged.groupby(['Building','Apt'])['LeaseStartDate'].transform('min')

    merged['Rent'] = merged.groupby(['Building','Apt'])['Rent'].transform('min')

    merged['BalAndBoth'] = merged.groupby(['Building','Apt'])['BalAndBoth'].transform('min')
    

In [10]:
def simplify_merged_table(merged):
    '''Simplify the merged table
    
    There are many columns left for different calculations, for this parts, keeps some columns:
    ['Agent_Combined','Manager_Combined','TL_Combined',\
    'Building','Apt','BalAndBoth','DealDate','LeaseStartDate','Rent','Comm']
    
    Delete duplicates then keep only one row for the same deal and sum up the comm.
    
    Args:
        merged: data C mapped with info from B + Combined supervisors info.
    
    Returns:
        merged_final: simplified data C mapped with info from B & Combined supervisors info
    '''
    
    merged_final_columns = merged[['Agent_Combined','Manager_Combined','TL_Combined','Building','Apt','BalAndBoth','DealDate','LeaseStartDate','Rent','Comm']]

    merged_final = merged_final_columns.copy()

    merged_final['Comm'] = merged_final_columns.groupby(['Building','Apt'])['Comm'].transform(sum)

    merged_final = merged_final.drop_duplicates().reset_index(drop = True)
    
    return merged_final

In [11]:
def building_stats(merged,merged_final):
    '''Build a table to sum up the stats for buildings in certain period
    
    This table contains total units number and clients number for different floor plan
    
    Args:
        merged: data C mapped with info from B
        merged_final: simplified data C mapped with info from B & Combined supervisors info
        
    Returns:
        add_up: building stats table contains units num and clients num
    '''
    
    people_num = merged[['Building','BalAndBoth']].\
                pivot_table(index = 'Building',columns = 'BalAndBoth',aggfunc = len, margins = True, fill_value = '')

    # units nunm
    units_num = merged_final[['Building','BalAndBoth']].\
                pivot_table(index = 'Building',columns = 'BalAndBoth',aggfunc = len, margins = True, fill_value = '')

    add_up = people_num.astype(str) +'/'+ units_num.astype(str)

    add_up = add_up.replace('\.0','',regex=True)

    add_up = add_up.replace(to_replace ='^/', value = '', regex = True)

    # mapping comm column
    for i,k in add_up.iterrows():
        if i != 'All':
            add_up.loc[i,'Comm Total'] = merged_final[merged_final['Building']  == i]['Comm'].sum()
        else:
            add_up.loc[i,'Comm Total'] = merged_final['Comm'].sum()
        
    add_up.sort_values( by = 'Comm Total', ascending = False, inplace = True)
    
    return add_up

In [12]:
def client_sources(merged):
    '''Calculate percentage of sources, work&schools, life stages of clients
    
    Args:
        merged: data C mapped with info from B
    
    Returns:
        sources_percentage: percentage for different sources type
        schools_percentage: percentage of diffrent schools or employed
        chools_percentage_without_work: percentage of diffrent schools only
        life_stage_percentage: percenrage of life stage of clients
    '''

    sources_percentage =pd.DataFrame(merged['Sources'].\
                                     value_counts(normalize = True, ascending = False).\
                                     round(4)*100).round(2).astype(str) + '%'

    schools_percentage = pd.DataFrame(merged['School'].\
                                      value_counts(normalize = True, ascending = False).\
                                      round(4)*100).round(2).astype(str) + '%'
    
    schools_percentage_without_work = pd.DataFrame(merged[merged['School'] !='employed']['School'].\
                                                   value_counts(normalize = True, ascending = False).\
                                                   round(4)*100).round(2).astype(str) + '%'

    life_stage_percentage = pd.DataFrame(merged['LifeStage'].\
                                         value_counts(normalize = True, ascending = False).\
                                         round(4)*100).round(2).astype(str) + '%'

    return sources_percentage, schools_percentage, schools_percentage_without_work, life_stage_percentage

In [13]:
def output_excels(output_type, file_name, merged_final, add_up, 
                  agent_rank, manager_rank, tl_rank,
                  sources_percentage, schools_percentage,schools_percentage_without_work, life_stage_percentage):
    
    '''Output the final excel report 
    
    Report contains different sheets, each sheets contains different stats. There will be 2 types
    of excel report
    "all": a report contains all the stats
    "main": a report only contains the main chart (deals chart)
    
    Args:
        output_type:
            "all": a report contains all the stats
            "main": a report only contains the main chart (deals chart)
        file_name: the report name
        merged_final: merged final DataFrame
        add_up: building stats
        agent_rank: agent commission ranking DataFrame
        manager_rank: manager team commission ranking DataFrame
        tl_rank: team leader team commission ranking DataFrame
        sources_percentage: percentage of sources of clients
        schools_percentage: percentage of diffrent schools or employed
        chools_percentage_without_work: percentage of diffrent schools only
        life_stage_percentage: percenrage of life stage of clients

    '''
    
    file_name = file_name + '.xlsx'
    
    if output_type == 'all':
        writer = pd.ExcelWriter(file_name,  datetime_format='MM-DD-YYYY')
        merged_final.to_excel(writer, sheet_name = 'main')
        add_up.to_excel(writer, sheet_name = 'building')
        agent_rank.to_excel(writer, sheet_name = 'agent ranking')
        manager_rank.to_excel(writer, sheet_name = 'manager ranking')
        tl_rank.to_excel(writer,sheet_name = 'team leader ranking')
        sources_percentage.to_excel(writer,sheet_name = 'sources percentage')
        schools_percentage.to_excel(writer,sheet_name = 'school percentage')
        schools_percentage_without_work.to_excel(writer,sheet_name = 'schools without_work')
        life_stage_percentage.to_excel(writer,sheet_name = 'life_stage_percentage')
        writer.close()
    elif output_type == 'main':
        writer = pd.ExcelWriter(file_name,  datetime_format='MM-DD-YYYY')
        merged_final.to_excel(writer, sheet_name = 'main')
        writer.close()

In [14]:
def output_mail_champ_address(mail_champ, merged, start_date, end_date ):
    '''Output mail champ address in Excel
    
    If mail champ address is requested, output it in Excel file
    
    Args:
        mail_champ: If 'Yes' then output
        merged: use the DataFrame merged
        start_date: start date of the mail champ address, inclusively
        end_date: end date of the mail champ address, inclusively
    '''
    if mail_champ == 'Yes':
        mail_champ_address = merged[['Building','Email','Name','LeaseStartDate','School','LifeStage']]
        mail_champ_address.to_excel('mailchimp {} to {}.xlsx'.format(start_date.date(),end_date.date()))
    else:
        pass
        

In [19]:
def main(start_date,end_date,
         file_date,
         file_name,
         output_type='aaa',
         targets='all', 
         status='Pending + Closed',
         mail_champ='No'):
    
    '''Main function for CRM clean
    
    Import team structure data and data sets downloaded from CRM
    Generating data report for reviewing and dealboard
    Generating a mail champ address excel based on request
    
    If the certain period of deals contains any agents name hasn't been updated into the team structure list, 
    nothing will be outputted until HR updates the team structure file
    
    Args:
        start_date: start date for the report, inclusively, format example '2020-12-31'
        end_date: end date for the report, inclusively, format example '2020-12-31'
        file_date: the data sets we are using are formatted like 'mm-dd-B/C.xlsx'
        file_name: name of report, format example '12-1 to 12-7.xlsx'
        output_type: different types of report, default: dont write to excel yet
        targets: default is all agent, specify a certain name if need
        status: default all the deals except dropped one
        mailchamp: whether output mail champ, default: not
    
    '''
    
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
       
    agent_level, main_sheet_b, main_sheet_c = read_files(file_date)
    
    date_fix(main_sheet_c,'DealDate')
    date_fix(main_sheet_b,'LeaseStartDate')
    date_fix(main_sheet_b,'LeaseEndDate')
    
    time_range = period(main_sheet_c,start_date,end_date)
    
    agents_need_updates = []
    for name in time_range['registrant'].unique():
        if name not in agent_level['Agent Name'].unique():
            agents_need_updates.append(name)
        else:
            continue
    
    if len(agents_need_updates) > 0:
        print(*agents_need_updates,' not in the team structure file, contact HR to update.', sep = ', ')
        
    else: 
        deals_status(status,time_range)

        if 'all' in targets:
            pass
        else:
            time_range = time_range[time_range['registrant'].isin(targets)]

        time_range_tag = time_range.copy()
        time_range_tag['TL'] = time_range.apply(TL_match,agent_level = agent_level,axis = 1)
        time_range_tag['M'] = time_range.apply(M_match,agent_level = agent_level,axis = 1)
        time_range_tag['SA'] = time_range.apply(SA_match,agent_level = agent_level,axis = 1)


        agent_rank,manager_rank,tl_rank = ranking(time_range_tag)

        merged = mapping_building(time_range_tag,main_sheet_b)

        merging_cap(merged)

        merged_final = simplify_merged_table(merged)

        add_up = building_stats(merged,merged_final)

        sources_percentage, schools_percentage, schools_percentage_without_work, life_stage_percentage = client_sources(merged)


        output_excels(output_type, file_name, merged_final, add_up, 
                      agent_rank, manager_rank, tl_rank,
                      sources_percentage, schools_percentage,schools_percentage_without_work, life_stage_percentage)


        output_mail_champ_address(mail_champ, merged, start_date, end_date)
